In [1]:
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten

Using TensorFlow backend.


In [2]:
# Create the model

classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [3]:
# Image augmentation

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [4]:
# Prepare the images

training_set = train_datagen.flow_from_directory('Dataset_Cat_Or_Dog/training_set_new',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('Dataset_Cat_Or_Dog/test_set_new',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 360 images belonging to 2 classes.
Found 155 images belonging to 2 classes.


In [5]:
# Train the classifier

classifier.fit_generator(training_set,
                         steps_per_epoch = 1500,
                         epochs = 1,
                         validation_data = test_set,
                         validation_steps = 600,
                         verbose = 1)

Epoch 1/1
1500/1500 [==============================] - 2795s 2s/step - loss: 0.2025 - accuracy: 0.9089 - val_loss: 2.2193 - val_accuracy: 0.6968


In [9]:
# Making new predictions

from keras.preprocessing import image

test_image = image.load_img('Dataset_Cat_Or_Dog/single_prediction/cat_or_dog_1.jpg', 
                            target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print('It is a', prediction)

It is a dog


In [13]:
# Save model for later use of application

# classifier.save('cat_or_dog_model.h5')

from keras.models import load_model

reconstructed_model = load_model('cat_or_dog_model.h5')

test_image = image.load_img('Dataset_Cat_Or_Dog/single_prediction/cat.4041.jpg', 
                            target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = reconstructed_model.predict(test_image)

if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print('It is a', prediction)

It is a cat
